# Customer lifetime value in a discrete-time contractual setting
*math and Python implementation*

## Motivating question
In a monthly/yearly (i.e., discrete-time) subscription business setting (i.e., contractual), based on our current customer’s characteristics, what is the expected value of a new customer?
- Note that our setting is a discrete-time contractual setting. If your business model is not subscription based (i.e., non-contractual), please check out the [lifetimes](https://lifetimes.readthedocs.io/en/latest/?badge=latest) package in Python.

## Assumptions 
- here we only focus on revenue and ignore the customer acquisition cost, and other costs.
    
    
## Equation 
(Fader, Peter, & Bruce (2007a))

The equation of the expected value of customer life time value is quite straightforward:

- $E(CLV) = \sum_{n=1}^{\infty} m \frac{s(t)}{(1+d)^t} $
    
- m: subscription rate

- s(t): survival function at time t   

- d: discount rate reflecting the time value of money
    
    

## Example
Here is an example from the paper Fader, Peter, & Bruce (2007b). Assume we have 1000 customers, at Year 1 (t0) we have all 1000 customers. At Year 2 (t1), only 631 customers are active. And at Year 5, only 326 customers are still active. Assume our subscription rate is $100/year and the discount rate is 10%.

- m = $100/year
- d = 10% 

| ID   | Year 1 | Year 2 | Year 3 | Year 4 | Year 5 |
|------|--------|--------|--------|--------|--------|
| 1    | 1      | 1      | 0      | 0      | 0      |
| 2    | 1      | 0      | 0      | 0      | 0      |
| 3    | 1      | 1      | 1      | 0      | 0      |
| ...  |        |        |        |        |        |
| 1000 | 1      | 0      | 0      | 0      | 0      |
|      | 1000   | 631    | 468    | 382    | 326    |


$ E(CLV) = 100 + 100\frac{0.631}{1.1} + 100\frac{0.468}{1.1^2} + 100\frac{0.382}{1.1^3} + 100\frac{0.326}{1.1^4} + 100\frac{s(5)}{1.1^5} + ... 100\frac{s(n)}{1.1^n} $

- Based on the CLV equation, we can fill in the data and rewrite the equation as shown above.
- Issue: the only issue is we do not know future survival rate. Thus, we can use the Geometric-beta model to model our data and predict the survival function.    
    
## Geometric-beta model
(Fader, Peter, & Bruce (2007a))    

### customer duration ~ geometric($\theta$)     
We assume that customer duration/lifetime follows a geometric distribution because customers can only churn once.

- probability of churn: $\theta$

- probability of retention: $1-\theta$

- probability of customer churn at time t: P(T=t|$\theta$) = $\theta(1-\theta)^{t-1}$    

- survival rate at time t: $S(t|\theta) = (1-\theta)^t$

- retention rate at time t: $r(t) = \frac{s(t)}{s(t-1)}$

### $\theta \sim beta(\alpha, \beta)$
We model the heterogeneity of 𝜃 as a beta distribution. We use beta distribution because it is bounded by the interval [0, 1], and it comes in all possible shapes.

- defined on the interval [0, 1] 

- $f(\theta|\alpha, \beta) = \frac{\theta^{\alpha-1}(1-\theta)^{\beta-1}}{B(\alpha,\beta)}$, $\alpha, \beta > 0$

### Geometric-beta distribution    
Then we can combine the geometric distribution and the beta distribution and calculate the joint distribution. And through some calculation of Beta functions and Gamma functions, we can calculate the probability of customer churn at any time t and also the retention rate at any time t.

- $p(T=t|\alpha, \beta) = \int_{0}^{1}p(T=t|\theta)f(\theta|\alpha, \beta)d\theta$   

    - $p(T=1|\alpha, \beta) = \frac{B(\alpha+1, \beta)}{B(\alpha, \beta)} = \frac{\Gamma(\alpha+1)\Gamma(\beta)\Gamma(\alpha+\beta)}{\Gamma(\alpha +\beta+1)\Gamma(\alpha)\Gamma(\beta)} = \frac {\alpha}{\alpha+\beta}$  
    - $p(T=t|\alpha, \beta) = \frac{\beta+t-2}{\alpha+\beta+t-1}p(T=t-1)$ for t>1   

- $r(t) = \frac{\beta+t-1}{\alpha+\beta+t-1}$

- Likelihood function: probability of losing $n_1$ customer at t1, losing $n_2$ customer at t2, ... and keep ${n-\sum{n_t}}$ customer at the end of the observed period. 
$L(\alpha, \beta|data) = p(T=1|\alpha,\beta)^{n_1}p(T=2|\alpha,\beta)^{n_2} ...p(T=n-1|\alpha,\beta)^{n_{(n-1)}} s(T=t|\alpha,\beta)^{n-\sum{n_t}}$   

- Log-likelihood function: $LL(\alpha, \beta|data)= ln(L(\alpha, \beta|data))$

### Find the maximum likelihood estimator for $\alpha$ and $\beta$

- In order to find the maximum likelihood estimator for our parameters, we then maximize the log-likelihood function or minimize the negative log-likelihood function. Through optimization procedure, we get the MLE values for 𝛼 and 𝛽.


## Calculate CLV 
### Given $\alpha$ and $\beta$, calculate survival rate for each time point
- $r(t) = \frac{\beta+t-1}{\alpha+\beta+t-1}$

- $s(t) = 1$ when t = 0 

- $s(t) = r(t)s(t-1)$ when t>0

### calculate CLV 
choose a reasonable large number for k 

- $E(CLV) = \sum_{n=1}^{k} m \frac{s(t)}{(1+d)^t} $

## Python implementation
### Maximum likelihood estimation
Here is my Python implementation for calculate the log likelihood function, and to optimize the negative log likelihood function. Note that we need to give the parameter an initial value as a starting point for optimization. In our example here, the initial values for 𝛼 and 𝛽 are both 1.  


In [ ]:
from scipy.optimize import minimize
from autograd import value_and_grad, hessian
import numpy as np
import pandas as pd
import hvplot.pandas

def _loglikelihood(params, n, active, return_s=False):
    """
    params: alpha and beta initial values    
    n: total number of people starting at t0
    actives: # of people stayed active at each time point starting at t1
    """
    t = list(range(1, len(active)+1))  # t: [ 1, 2, ...]
    alpha, beta = params
    p = []
    s = []
    lost = []
    ll = []
    for i in t:
        if i == 1: 
            p.append(alpha/(alpha+beta))
            s.append(1- p[0])
            lost.append(n-active[0])
        else: 
            p.append((beta+i-2)/(alpha+beta+i-1) * p[-1])
            s.append(s[-1] - p[i-1])
            lost.append(active[i-2] - active[i-1])
            
        ll.append(lost[i-1] * np.log(p[i-1]))
        #print('p: ', p[i-1])
    ll.append(active[-1] * np.log(s[-1]))
    
    #print('s: ', s[-1])
    if return_s== True:
        return s
    else:
        return ll 
    
def _negative_log_likelihood(params, n, active):
    return -(np.sum(_loglikelihood(params, n, active)))       

In [ ]:
params = np.array((1,1)) #init params
n = 1000
active = [631, 482, 382, 326]
#active = [869, 743, 653, 593, 551, 517, 491]


In [ ]:
_negative_log_likelihood(params, n, active)

In [ ]:
res = minimize(
        _negative_log_likelihood,
        args = (n, active), #parameters we do not optimize on 
        tol=1e-13,
        x0= np.array((1,1)), #starting value of params 
        bounds=[(0, None), (0, None)],
        options={'ftol' : 1e-100000000},
    )
res

Through optimization, the optimized negative log likelihood is 1409.5, and the MLE for 𝛼 and 𝛽 are 0.78 and 1.35.

### Model fit

Here is my function to access model fit. Here I am only comparing the observed and calculated survival rate. You can change this function to compare likelihood function and retention rate as well.
The first plot here used the initial parameters (1,1) as the model parameter, we can see that the model calculated survival rate is not very ideal. The second plot used the MLE values for the parameters and the model calculated survival rate is very close to the observed survival rate.


In [ ]:
def model_fit_survival(params, n, active):
    t = list(range(1, len(active)+1))
    df_plot = pd.DataFrame({'t': t, 
                            'observed': [x/n for x in active], 
                            'model': _loglikelihood(params, n, active, return_s=True)})
    return df_plot.hvplot('t',['observed', 'model'], title = 'Survival rate')


In [ ]:
# using init params (1,1)
model_fit_survival(params, n, active)

In [ ]:
# using optimized params res.x 
model_fit_survival(res.x, n, active)

### Calculate CLV
Now we can calculate the expected value of the customer lifetime value using the MLE parameters. We can see that the expected value of a new customer is $362.


In [ ]:
def e_clv(alpha, beta, d, net_cf):
    t = list(range(0, 200))
    r = []
    s = []
    disc = []
    for i in t:
        if i == 0: 
            r.append(0)
            s.append(1)
            disc.append(1)
        else:
            r.append((beta+i-1)/(alpha+beta+i-1))
            s.append(r[i]*s[i-1])
            disc.append((1/(1+d)**i))
    clv = net_cf * sum([x * y for x, y in zip(s, disc)])
    return clv

        
    

In [ ]:
alpha, beta = res.x
d = 0.1
net_cf = 100
e_clv(alpha, beta, d, net_cf)

By Sophia Yang on [August 24, 2020](https://towardsdatascience.com/customer-lifetime-value-in-a-discrete-time-contractual-setting-math-and-python-implementation-af3ef606cefe)